In [29]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

def scrape_season(start_year, end_year):

    season_year = {}

    for year in range(start_year,end_year):

        #print('Get the colleges from {year} year'.format(year = year))

        url = 'https://www.sports-reference.com/cbb/seasons/{year}.html'.format(year = str(year))

        # collect HTML data
        html = urlopen(url)
                
        # create beautiful soup object from HTML
        soup = BeautifulSoup(html, features="html")

        # get table
        table = soup.find('table', id='conference-summary')

        # Collecting Data
        conference_data = []
        for row in table.tbody.find_all('tr'):    
            # Find all data for each column
            rows = row.find_all('td')
            conference_data.append(row)

        rows_data = [[td.getText().replace('-', ' ') for td in conference_data[i].findAll('td')]
                            for i in range(len(conference_data))]

        conference_list =[]

        for conference in rows_data[0:]:
            conference_list.append(conference[0])

        hyperlinks_dict = {}
        for link in table.tbody.find_all('a'):
            if link.get_text() in conference_list:
                hyperlinks_dict[link.get_text()] = link.get('href')

        
        season_dict = {}
        season_dict['season'] = hyperlinks_dict
        season_year[str(year)] = season_dict

    return season_year







In [30]:
start_year = 2021
end_year = 2022

season_links = scrape_season(start_year, end_year)

In [50]:
def find_colleges_links(season_links):

    for year in season_links:

        for college in season_links[year]['season']:

            url = season_links[year]['season'][college]

            # collect HTML data
            root_link = 'https://www.sports-reference.com'
            url = root_link + url
            html = urlopen(url)
                    
            # create beautiful soup object from HTML
            soup = BeautifulSoup(html, features="html")

            # get table
            table = soup.find('table', id='standings')

            # Collecting Data
            college_data = []
            for row in table.tbody.find_all('tr'):    
                # Find all data for each column
                rows = row.find_all('td')
                college_data.append(row)

            rows_data = [[td.getText().replace('-', ' ') for td in college_data[i].findAll('td')]
                                for i in range(len(college_data))]

            college_list =[]

            for college in rows_data[0:]:
                college_list.append(college[0])

            hyperlinks_dict = {}
            for link in table.tbody.find_all('a'):
                if link.get_text() in college_list:
                    hyperlinks_dict[link.get_text()] = link.get('href')


            season_links[year]['college'] = hyperlinks_dict
  

    return season_links

            


In [51]:
global_dict = find_colleges_links(season_links)

In [52]:
global_dict['2021']['college']

{'Brown': '/cbb/schools/brown/2021.html',
 'Columbia': '/cbb/schools/columbia/2021.html',
 'Cornell': '/cbb/schools/cornell/2021.html',
 'Dartmouth': '/cbb/schools/dartmouth/2021.html',
 'Harvard': '/cbb/schools/harvard/2021.html',
 'Pennsylvania': '/cbb/schools/pennsylvania/2021.html',
 'Princeton': '/cbb/schools/princeton/2021.html',
 'Yale': '/cbb/schools/yale/2021.html'}